In [1]:
from datetime import datetime
print(f"统计于 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
from show_result import show_result
import os
for f in os.listdir("results/tests_1015")[::-1][:3]:
    print("\033[32m> {}\033[0m".format(f))
    show_result(f"results/tests_1015/{f}")
    print()

统计于 2024-10-15 13:02:52
> exp_3_missing_nodes_80_mask_tcn_mlp

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[    dynst] 4.913	-	-	24.86	4.705	-	-	27.289	5.711	-	-	30.573
[    epoch] 390	-	-	8	291	-	-	6	249	-	-	4

> exp_2_missing_nodes_80_mask_mlp

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[    dynst] 4.513	-	-	24.456	4.403	-	-	27.062	5.306	-	-	31.879
[    epoch] 212	-	-	8	225	-	-	6	327	-	-	5

> exp_1_missing_nodes_80_mask_tcn

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[    dynst] 4.729	-	-	24.765	5.151	-	-	31.401	5.784	-	-	32.448
[    epoch] 493	-	-	495	311	-	-	489	384	-	-	5



False

In [ ]:
raise NotImplementedError
import os

def delete_pth_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.pth'):
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Deleted: {file_path}")

# 使用示例
delete_pth_files('results/tests_old')
